In [11]:
import numpy as np
import pandas as pd

pd.set_option('display.max_colwidth', None)

file = "newORfail.csv"
df = pd.read_csv(file)
# df.head()

for index, row in df.iterrows():

    real_output = eval(str(row['real_output']))
    df.at[index, 'Real Output'] = real_output['n200']

    gold_output = eval(str(row['gold_output']))
    df.at[index, 'Gold Output'] = gold_output['n200']
    
    pv = str(row['MC']).split(',')
    df.at[index, 'Process Variation'] = pv[1]

    df.at[index, 'Equal?'] = (gold_output['n200'] == real_output['n200'])

drop_col = ['MC', 'real_output', 'gold_output']
df = df.drop(drop_col, axis=1)

df = df[df['Equal?'] == False]
# df.head(10)

variations = df['Process Variation'].unique()


C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_16944\1614029846.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '00010101000001001001010020020020010020020020010020020020' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Real Output'] = real_output['n200']
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_16944\1614029846.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '00010101000001001001010011011011010011011011010011011011' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Gold Output'] = gold_output['n200']
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_16944\1614029846.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-0.09' has dtype incompatible

In [12]:
from functions_file import *

pd.set_option('display.max_columns', None)
fault_distribution = pd.DataFrame()

for pv in variations:

    sample = df[df['Process Variation'] == pv]


    for index, row in sample.iterrows():
        realOutput = str(row['Real Output'])
        goldOutput = str(row['Gold Output'])
        inputPattern = str(row['pattern']).replace(",", "")

        sample.loc[index, 'SA0'] = 1 if O_SA0(realOutput) else 0
        sample.at[index, 'SA1'] = 1 if O_SA1(realOutput) else 0
        sample.at[index, 'EscapeRight'] = 1 if outputEscapeRight(goldOutput, realOutput) else 0
        sample.at[index, 'EscapeLeft'] = 1 if outputEscapeLeft(goldOutput, realOutput) else 0
        sample.at[index, 'Overflow'] = 1 if overflow(realOutput) else 0
        sample.at[index, 'AND->OR'] = 1 if AND_OR(inputPattern, realOutput) else 0
        sample.at[index, 'OR->AND'] = 1 if OR_AND(inputPattern, realOutput) else 0
        sample.at[index, 'alwaysFail_00'] = 1 if alwaysFail_00(inputPattern, realOutput, goldOutput) else 0
        sample.at[index, 'alwaysFail_01'] = 1 if alwaysFail_01(inputPattern, realOutput, goldOutput) else 0
        sample.at[index, 'alwaysFail_10'] = 1 if alwaysFail_10(inputPattern, realOutput, goldOutput) else 0
        sample.at[index, 'alwaysFail_11'] = 1 if alwaysFail_11(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'oftenFail_00'] = 1 if oftenFail_00(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'oftenFail_01'] = 1 if oftenFail_01(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'oftenFail_10'] = 1 if oftenFail_10(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'oftenFail_11'] = 1 if oftenFail_11(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'neverFail_00'] = 1 if neverFail_00(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'neverFail_01'] = 1 if neverFail_01(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'neverFail_10'] = 1 if neverFail_10(inputPattern, realOutput, goldOutput) else 0
        # sample.at[index, 'neverFail_11'] = 1 if neverFail_11(inputPattern, realOutput, goldOutput) else 0

        for sequence in sequences:
            if alwaysFail_sequence(inputPattern, realOutput, goldOutput, sequence) is True:
                sample.at[index, "alwaysFail_" + str(sequence)] = 1
            else:
                sample.at[index, "alwaysFail_" + str(sequence)] = 0
        

    sample['totalTemplates'] = sample.iloc[:, 6:sample.shape[1]-1].sum(axis=1)

    new_row = {}
    for col in sample.columns:
        if pd.api.types.is_numeric_dtype(sample[col]):
            new_row[col] = sample[col].sum()
        else:
            new_row[col] = sample[col].iloc[-1]

    
    sample = pd.concat([sample, pd.DataFrame([new_row], columns=sample.columns)], ignore_index=True)

    print("Process Variation: ", pv)
    display(sample.head())
    display(sample.tail())

    fault_distribution = pd.concat([fault_distribution, pd.DataFrame([sample.iloc[-1]], columns=sample.columns)], ignore_index=True)

C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_16944\3289478364.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.loc[index, 'SA0'] = 1 if O_SA0(realOutput) else 0
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_16944\3289478364.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.at[index, 'SA1'] = 1 if O_SA1(realOutput) else 0
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_16944\3289478364.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

Process Variation:  -0.09


C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_16944\3289478364.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['totalTemplates'] = sample.iloc[:, 6:sample.shape[1]-1].sum(axis=1)


,config,pattern,Real Output,Gold Output,Process Variation,Equal?,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111,totalTemplates
0,and_spb_0_and_spb_0_or_spa_0_and,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00010101000001001001010020020020010020020020010020020020,00010101000001001001010011011011010011011011010011011011,-0.09,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,and_spb_0_and_spb_0_or_spa_0_and,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00101010000010010010100110110110100110110110100110110110,00010101000001001001010011011011010011011011010011011011,-0.09,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,and_spb_0_and_spb_0_or_spa_0_and,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00101011000010010010100110110110100110110110100110110110,00010101000001001001010011011011010011011011010011011011,-0.09,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,and_spb_0_and_spb_0_or_spa_0_and,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",33323333322233232232322322332333332322332333322322341322,00010101000001001001010011011011010011011011010011011011,-0.09,False,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,and_spb_0_and_spb_0_or_spa_0_and,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000001000000000001000010011011010011010011010010010011,00010101000001001001010011011011010011011011010011011011,-0.09,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,config,pattern,Real Output,Gold Output,Process Variation,Equal?,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111,totalTemplates
1763,xor_spb_2_xor_spb_2_or_spa_2_or,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000001000000000001000000000001000000000001010010010011,00010101000001001001010011011011010011011011010011011011,-0.09,False,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1764,xor_spb_2_xor_spb_2_or_spa_2_xor,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",11312123212123122121231222331232222222222332221222222221,00010101000001001001010011011011010011011011010011011011,-0.09,False,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1765,xor_spb_2_xor_spb_2_or_spa_2_xor,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",32322234234342234232223432342443432443222322433343332322,00010101000001001001010011011011010011011011010011011011,-0.09,False,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1766,xor_spb_2_xor_spb_2_or_spa_2_xor,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000001000000000001000000000001000000000001010010010011,00010101000001001001010011011011010011011011010011011011,-0.09,False,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1767,xor_spb_2_xor_spb_2_or_spa_2_xor,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",00000001000000000001000000000001000000000001010010010011,00010101000001001001010011011011010011011011010011011011,-0.09,False,50.0,0.0,163.0,128.0,398.0,0.0,180.0,0.0,105.0,110.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,30.0,0.0,1195.0


...

C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_16944\3289478364.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.loc[index, 'SA0'] = 1 if O_SA0(realOutput) else 0
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_16944\3289478364.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.at[index, 'SA1'] = 1 if O_SA1(realOutput) else 0
C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_16944\3289478364.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

Process Variation:  0.08


C:\Users\Carrie Lei\AppData\Local\Temp\ipykernel_16944\3289478364.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['totalTemplates'] = sample.iloc[:, 6:sample.shape[1]-1].sum(axis=1)


,config,pattern,Real Output,Gold Output,Process Variation,Equal?,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111,totalTemplates
0,and_spb_0_and_spb_0_or_spa_0_and,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",01010101010101011011010111011011010111011011010111011011,00010101000001001001010011011011010011011011010011011011,0.08,False,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1,and_spb_0_and_spb_0_or_spa_0_dff,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",01010101010101011011010111011011010111011011010111011011,00010101000001001001010011011011010011011011010011011011,0.08,False,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
2,and_spb_0_and_spb_0_or_spa_0_inv,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",01010101010101011011010111011011010111011011010111011011,00010101000001001001010011011011010011011011010011011011,0.08,False,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,and_spb_0_and_spb_0_or_spa_0_or,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",01010101010101011011010111011011010111011011010111011011,00010101000001001001010011011011010011011011010011011011,0.08,False,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,and_spb_0_and_spb_0_or_spa_0_xor,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",01010101010101011011010111011011010111011011010111011011,00010101000001001001010011011011010011011011010011011011,0.08,False,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


,config,pattern,Real Output,Gold Output,Process Variation,Equal?,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111,totalTemplates
1410,xor_spb_2_xor_spb_2_or_spa_2_or,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",52445344233534344433444434443443443443525243524443344444,00010101000001001001010011011011010011011011010011011011,0.08,False,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1411,xor_spb_2_xor_spb_2_or_spa_2_xor,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",10111111010111011011110111121011110111121011110111111111,00010101000001001001010011011011010011011011010011011011,0.08,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1412,xor_spb_2_xor_spb_2_or_spa_2_xor,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",01010101010101011011010111011011010111011011010111011011,00010101000001001001010011011011010011011011010011011011,0.08,False,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1413,xor_spb_2_xor_spb_2_or_spa_2_xor,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",52444344444444434434433444452443352443444335243443344343,00010101000001001001010011011011010011011011010011011011,0.08,False,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1414,xor_spb_2_xor_spb_2_or_spa_2_xor,"00,00,00,01,00,10,00,11,00,00,00,00,00,01,00,00,10,00,00,11,00,01,00,00,01,01,00,01,10,00,01,11,00,10,00,00,10,01,00,10,10,00,10,11,00,11,00,00,11,01,00,11,10,00,11,11",52444344444444434434433444452443352443444335243443344343,00010101000001001001010011011011010011011011010011011011,0.08,False,125.0,0.0,498.0,0.0,141.0,227.0,0.0,227.0,0.0,0.0,0.0,210.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1428.0


In [13]:
drop_cols = ['config', 'pattern', 'Real Output', 'Gold Output', 'Equal?']
fault_distribution = fault_distribution.drop(drop_cols, axis=1).sort_values(by='Process Variation')
fault_distribution

,Process Variation,SA0,SA1,EscapeRight,EscapeLeft,Overflow,AND->OR,OR->AND,alwaysFail_00,alwaysFail_01,alwaysFail_10,alwaysFail_11,alwaysFail_0000,alwaysFail_0001,alwaysFail_0010,alwaysFail_0011,alwaysFail_0100,alwaysFail_0101,alwaysFail_0110,alwaysFail_0111,alwaysFail_1000,alwaysFail_1001,alwaysFail_1010,alwaysFail_1011,alwaysFail_1100,alwaysFail_1101,alwaysFail_1110,alwaysFail_1111,totalTemplates
6,-0.0,125.0,0.0,308.0,0.0,126.0,6.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,576.0
15,-0.01,30.0,0.0,306.0,0.0,316.0,0.0,85.0,0.0,60.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,892.0
7,-0.02,255.0,0.0,434.0,0.0,769.0,0.0,0.0,0.0,15.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,20.0,0.0,1528.0
14,-0.03,35.0,0.0,387.0,0.0,167.0,0.0,80.0,0.0,85.0,90.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,55.0,60.0,0.0,962.0
12,-0.04,30.0,0.0,469.0,0.0,360.0,0.0,89.0,0.0,45.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,0.0,1063.0
2,-0.05,50.0,0.0,279.0,125.0,731.0,6.0,5.0,6.0,85.0,75.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,10.0,0.0,1392.0
1,-0.06,0.0,0.0,691.0,27.0,867.0,5.0,10.0,5.0,75.0,75.0,3.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,15.0,0.0,5.0,10.0,10.0,0.0,1813.0
3,-0.07,50.0,0.0,360.0,84.0,316.0,20.0,230.0,20.0,135.0,140.0,11.0,0.0,0.0,0.0,2.0,5.0,20.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,5.0,1413.0
9,-0.08,225.0,0.0,390.0,500.0,54.0,0.0,305.0,0.0,200.0,205.0,35.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,35.0,40.0,0.0,2029.0
0,-0.09,50.0,0.0,163.0,128.0,398.0,0.0,180.0,0.0,105.0,110.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,30.0,0.0,1195.0


fault_distribution_ratio = fault_distribution.iloc[:, 1:-1].div(fault_distribution['totalTemplates'], axis=0)
fault_distribution_ratio.insert(0, 'Process Variation', fault_distribution['Process Variation'])
fault_distribution_ratio
